In [ ]:
# Machine Learning

In [ ]:
import numpy as np
import time
import PIL.Image as Image
import matplotlib.pylab as plt
import zipfile
import os
import shutil
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import datetime
%load_ext tensorboard

In [ ]:
# Extracting images

parent_dir = "/content"
cat_breeds_dir = "cat-dataset"
path = os.path.join(parent_dir, cat_breeds_dir)

if os.path.exists(path):
  shutil.rmtree(path)

if os.path.exists(path) == False:
  os.mkdir(path)
  cat_breeds_zip = zipfile.ZipFile("/content/drive/MyDrive/dataset-final.zip")
  cat_breeds_zip.extractall(path)
  cat_breeds_zip.close()
  print("Extract success!")


In [ ]:
PATH = os.path.join(path, "dataset-final")

cat_dir = os.path.join(PATH)

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  cat_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  cat_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

In [ ]:
class_names = np.array(train_ds.class_names)
print(class_names)

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) 
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)) 

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
classifier_model = mobilenet_v2

In [ ]:
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_model, input_shape=IMAGE_SHAPE+(3,))
])

In [ ]:
result_batch = classifier.predict(train_ds)

In [ ]:
predicted_class_names = class_names
predicted_class_names

In [ ]:
plt.figure(figsize=(20,20))
plt.subplots_adjust(hspace=0.5)

for n in range(8):
  plt.subplot(2,4,n+1)
  plt.imshow(image_batch[n].numpy())
  plt.title(predicted_class_names[n])
  plt.axis('off')
_ = plt.suptitle("Cat Breeds predictions")

In [ ]:
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

feature_extractor_model = mobilenet_v2

In [ ]:
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=False)

In [ ]:
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)

In [ ]:
num_classes = len(class_names)

model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(num_classes)
])

model.summary()

In [ ]:
predictions = model(image_batch)
predictions.shape

In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1)

In [ ]:
NUM_EPOCHS = 20

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=NUM_EPOCHS,
                    callbacks=tensorboard_callback)

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
predicted_batch = model.predict(image_batch)
predicted_id = tf.math.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]
print(predicted_label_batch)

In [ ]:
plt.figure(figsize=(20,20))
plt.subplots_adjust(hspace=0.0, wspace=0.5)
for images, labels in val_ds.take(1):
  classifications = model(images)
  
  for i in range(8):
    ax = plt.subplot(2, 8, i + 1)
    plt.imshow(images[i].numpy())
    index = np.argmax(classifications[i])
    plt.title("Pred: " + class_names[index] + "\nReal: " + class_names[labels[i]])

In [ ]:
from google.colab import files

uploaded = files.upload()
uploaded_filename = ""

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  uploaded_filename = fn

image_location = os.path.join(parent_dir, uploaded_filename)

image = tf.io.read_file(image_location)
image = tf.io.decode_image(image, channels=3, dtype=tf.dtypes.float32)
image = tf.image.resize(image, [224, 224])
image = tf.expand_dims(image, axis=0)

predicted = model.predict(image)
predicted_id = tf.math.argmax(predicted, axis=-1)
predicted_label = class_names[tuple(predicted_id)]
print()
print("Predict Result : ")
print(predicted_label)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
  f.write(tflite_model)